In [35]:
from keras.preprocessing.text import Tokenizer
from keras.datasets import reuters
import numpy as np 
max_words = 1000

print('Loading data...')
(x, y), (_, _) = reuters.load_data(num_words=max_words, test_split=0.)
print(len(x), 'train sequences')

num_classes = np.max(y) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x = tokenizer.sequences_to_matrix(x, mode='binary')
print('x_train shape:', x.shape)


Loading data...
11228 train sequences
46 classes
Vectorizing sequence data...
x_train shape: (11228, 1000)


In [36]:
n_clusters = len(np.unique(y))
n_clusters

46

In [37]:
from keras.optimizers import SGD
from keras.initializers import VarianceScaling
from utils.dec import DEC
init = 'glorot_uniform'
pretrain_optimizer = 'adam'
update_interval = 30
pretrain_epochs = 10000
dims=[x.shape[-1], 500, 500, 2000, 46]
init = VarianceScaling(scale=1. / 3., mode='fan_in',distribution='uniform')  # [-limit, limit], limit=sqrt(1./fan_in)
pretrain_optimizer = SGD(learning_rate=1, momentum=0.9)
dec = DEC(dims=dims, n_clusters=n_clusters, initializer=init)
dec.pretrain(x=x, optimizer=pretrain_optimizer,epochs=pretrain_epochs)
dec.compile(optimizer=SGD(0.01, 0.9), loss='kld')
dec.fit(x,update_interval=update_interval)

C:\Users\vioritiva\AppData\Roaming\Python\Python311\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Pretraining autoencoder model...
Epoch 1/10000
44/44 [==============================] - 5s 73ms/step - loss: 0.0537
Epoch 2/10000
44/44 [==============================] - 3s 60ms/step - loss: 0.0462
Epoch 3/10000
44/44 [==============================] - 2s 52ms/step - loss: 0.0453
Epoch 4/10000
44/44 [==============================] - 2s 48ms/step - loss: 0.0430
Epoch 5/10000
44/44 [==============================] - 2s 51ms/step - loss: 0.0421
Epoch 6/10000
44/44 [==============================] - 2s 49ms/step - loss: 0.0417
Epoch 7/10000
44/44 [==============================] - 2s 53ms/step - loss: 0.0415
Epoch 8/10000
44/44 [==============================] - 2s 48ms/step - loss: 0.0414
Epoch 9/10000
44/44 [==============================] - 2s 48ms/step - loss: 0.0412
Epoch 10/10000
44/44 [==============================] - 2s 48ms/step - loss: 0.0411
Epoch 11/10000
44/44 [==============================] - 2s 47ms/step - loss: 0.0409
Epoch 12/10000
44/44 [==============================

In [ ]:
y_pred = dec.predict(x)

In [ ]:
import numpy as np
relation = dict((x, 0) for x in range(n_clusters))
for i in range(n_clusters):
    u, indeces = np.unique(y_pred[y == i], return_inverse=True)
    i_pred = u[np.argmax(np.bincount(indeces))]
    relation[i_pred] = i
    
relation

{0: 0,
 1: 0,
 2: 40,
 3: 39,
 4: 0,
 5: 3,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 42,
 14: 0,
 15: 27,
 16: 0,
 17: 28,
 18: 41,
 19: 0,
 20: 21,
 21: 23,
 22: 0,
 23: 36,
 24: 0,
 25: 0,
 26: 0,
 27: 20,
 28: 0,
 29: 0,
 30: 14,
 31: 0,
 32: 16,
 33: 32,
 34: 0,
 35: 44,
 36: 0,
 37: 0,
 38: 45,
 39: 0,
 40: 0,
 41: 19,
 42: 0,
 43: 11,
 44: 43,
 45: 38}

In [ ]:
y_pred_corr = np.array(list(map(lambda x: relation[x], y_pred)))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred_corr)

0.2300498753117207

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

normalized_mutual_info_score(y, y_pred_corr)

0.3702891647841177

In [ ]:
from sklearn.metrics import silhouette_score
# Calculate the silhouette score
silhouette_avg = silhouette_score(x, y_pred)

print("The average silhouette_score is :", silhouette_avg)

The average silhouette_score is : -0.015748422650680025
